# Ensemble Learning

In [113]:
# The following is a script to:
# 1. 

In [1]:
# Initial imports
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from matplotlib import pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split

from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier

## Read the CSV and Perform Basic Data Cleaning

In [2]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Encoding

Need to encode categorical data into integers for training, testing, and predictions

In [3]:
# First will need to encode two columns with strings for the month and year: issue_d and next_pymnt_d
# Create dictionary with #s for months to replace in the existing df
name_to_num = {
    "Jan" : 1,
    'Feb': 2,
    'Mar': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Jul': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'Nov': 11,
    'Dec': 12}

In [4]:
#First break the issue_d column into two and separate the month from the year - save each of these in a separate dataframe
issue_df= df["issue_d"].str.split("-",expand=True)
issue_df.rename(columns={0: "issue_month", 1: "issue_year"},inplace=True)

next_pymnt_df= df["next_pymnt_d"].str.split("-",expand=True)
next_pymnt_df.rename(columns={0: "next_pymnt_month", 1: "next_pymnt_year"},inplace=True)

next_pymnt_df.head()

,next_pymnt_month,next_pymnt_year
0,May,2019
1,May,2019
2,May,2019
3,May,2019
4,May,2019


In [5]:
# Encode the month name in both of the dataframes using the dictionary of months encoded to integers
issue_df["issue_month"] = issue_df["issue_month"].apply(lambda x: name_to_num[x])
next_pymnt_df["next_pymnt_month"] = next_pymnt_df["next_pymnt_month"].apply(lambda x: name_to_num[x])
next_pymnt_df.head()

,next_pymnt_month,next_pymnt_year
0,5,2019
1,5,2019
2,5,2019
3,5,2019
4,5,2019


In [6]:
# Concatenate the issue month and issue year dataframes with the original and drop the original columns with the string issue date in a single column
df = pd.concat([df,issue_df,next_pymnt_df],axis="columns", join='inner',)
df.rename(columns={0: "issue_month", 1: "issue_year"},inplace=True)
df.drop(columns=["issue_d","next_pymnt_d"],inplace=True)
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issue_month,issue_year,next_pymnt_month,next_pymnt_year
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,low_risk,n,27.24,0.0,...,65687.0,38199.0,2000.0,61987.0,N,N,3,2019,5,2019
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,low_risk,n,20.23,0.0,...,271427.0,60641.0,41200.0,49197.0,N,N,3,2019,5,2019
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,low_risk,n,24.26,0.0,...,60644.0,45684.0,7500.0,43144.0,N,N,3,2019,5,2019
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,low_risk,n,31.44,0.0,...,99506.0,68784.0,19700.0,76506.0,N,N,3,2019,5,2019
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,low_risk,n,18.76,0.0,...,219750.0,25919.0,27600.0,20000.0,N,N,3,2019,5,2019


In [7]:
# Next, use LabelEncoder to convert specific columns to binary integers 0 & 1
# Create the LabelEncoder instance
le = LabelEncoder()

In [8]:
# Fit and encode the necessary df columns (i.e., those that binary - only 2 options) with the LabelEncoder
# Encode the verification_status column

le.fit(df["verification_status"])
df["verification_status"] = le.transform(df["verification_status"])

# Encode the loan_status column
le.fit(df["loan_status"])
df["loan_status"] = le.transform(df["loan_status"])

# Encode the pymnt_plan column
le.fit(df["pymnt_plan"])
df["pymnt_plan"] = le.transform(df["pymnt_plan"])

# Encode the initial_list_status column
le.fit(df["initial_list_status"])
df["initial_list_status"] = le.transform(df["initial_list_status"])

# Encode the application_type column
le.fit(df["application_type"])
df["application_type"] = le.transform(df["application_type"])

# Encode the hardship_flag column
le.fit(df["hardship_flag"])
df["hardship_flag"] = le.transform(df["hardship_flag"])

# Encode the debt_settlement_flag column
le.fit(df["debt_settlement_flag"])
df["debt_settlement_flag"] = le.transform(df["debt_settlement_flag"])

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issue_month,issue_year,next_pymnt_month,next_pymnt_year
0,10500.0,0.1719,375.35,RENT,66000.0,1,1,0,27.24,0.0,...,65687.0,38199.0,2000.0,61987.0,0,0,3,2019,5,2019
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,2,1,0,20.23,0.0,...,271427.0,60641.0,41200.0,49197.0,0,0,3,2019,5,2019
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,2,1,0,24.26,0.0,...,60644.0,45684.0,7500.0,43144.0,0,0,3,2019,5,2019
3,10000.0,0.1640,353.55,RENT,92000.0,2,1,0,31.44,0.0,...,99506.0,68784.0,19700.0,76506.0,0,0,3,2019,5,2019
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,0,1,0,18.76,0.0,...,219750.0,25919.0,27600.0,20000.0,0,0,3,2019,5,2019


In [9]:
# Encode the home_ownership column using the get_dummies function since four potential strings populate this column's rows
df = pd.get_dummies(df, columns=["home_ownership"])
df.head()

,loan_amnt,int_rate,installment,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,hardship_flag,debt_settlement_flag,issue_month,issue_year,next_pymnt_month,next_pymnt_year,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
0,10500.0,0.1719,375.35,66000.0,1,1,0,27.24,0.0,0.0,...,0,0,3,2019,5,2019,0,0,0,1
1,25000.0,0.2000,929.09,105000.0,2,1,0,20.23,0.0,0.0,...,0,0,3,2019,5,2019,0,1,0,0
2,20000.0,0.2000,529.88,56000.0,2,1,0,24.26,0.0,0.0,...,0,0,3,2019,5,2019,0,1,0,0
3,10000.0,0.1640,353.55,92000.0,2,1,0,31.44,0.0,1.0,...,0,0,3,2019,5,2019,0,0,0,1
4,22000.0,0.1474,520.39,52000.0,0,1,0,18.76,0.0,1.0,...,0,0,3,2019,5,2019,0,1,0,0


## Split the Data into Training and Testing

In [10]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"].to_frame()

In [11]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,...,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issue_month,next_pymnt_month,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,6.881700e+04,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,0.669994,0.0,21.778153,0.217766,0.497697,12.587340,...,29734.128558,5.572240e+04,0.0,0.0,1.726172,4.616839,0.009285,0.526309,0.106747,0.357659
std,10277.348590,0.048130,288.062432,1.155800e+05,0.719105,0.0,20.199244,0.718367,0.758122,6.022869,...,26795.394232,5.095845e+04,0.0,0.0,0.743862,0.486161,0.095914,0.499311,0.308793,0.479314
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.0,0.000000,0.000000,0.000000,2.000000,...,100.000000,1.270000e+02,0.0,0.0,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,0.000000,0.0,13.890000,0.000000,0.000000,8.000000,...,11600.000000,2.288000e+04,0.0,0.0,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,1.000000,0.0,19.760000,0.000000,0.000000,11.000000,...,22100.000000,4.200000e+04,0.0,0.0,2.000000,5.000000,0.000000,1.000000,0.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,1.000000,0.0,26.660000,0.000000,1.000000,16.000000,...,39300.000000,7.249900e+04,0.0,0.0,2.000000,5.000000,0.000000,1.000000,0.000000,1.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,2.000000,0.0,999.000000,18.000000,5.000000,72.000000,...,509400.000000,1.426964e+06,0.0,0.0,3.000000,5.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
# Check the balance of our target values - integer of 1 = low risk and 0 = high risk 
y["loan_status"].value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [13]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [14]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [15]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [16]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [24]:
# Resample the training data with the BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=1)

In [18]:
# Calculated the balanced accuracy score
brf_y_pred = brf.predict(X_test_scaled)
balanced_accuracy_score(y_test, brf_y_pred)

0.7526743787568654

In [19]:
# Display the confusion matrix
confusion_matrix(y_test, brf_y_pred)

array([[   62,    39],
       [ 1856, 15248]], dtype=int64)

In [20]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, brf_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      0.61      0.89      0.06      0.74      0.53       101
          1       1.00      0.89      0.61      0.94      0.74      0.56     17104

avg / total       0.99      0.89      0.62      0.94      0.74      0.56     17205



# Need to add importances along with the labels

In [22]:
# List the features sorted in descending order (using [::-1]) by feature importance
feature_importances = brf.feature_importances_
sorted_indices = np.argsort(feature_importances)[::-1]
print(X_train.columns[sorted_indices], sep = "\n")

Index(['total_rec_prncp', 'last_pymnt_amnt', 'total_pymnt_inv', 'total_pymnt',
       'total_rec_int', 'int_rate', 'issue_month', 'mths_since_recent_inq',
       'installment', 'dti', 'out_prncp_inv', 'max_bal_bc', 'out_prncp',
       'revol_bal', 'mths_since_rcnt_il', 'avg_cur_bal', 'tot_hi_cred_lim',
       'annual_inc', 'mo_sin_old_rev_tl_op', 'total_bal_il', 'total_bc_limit',
       'il_util', 'bc_util', 'mo_sin_old_il_acct', 'bc_open_to_buy',
       'tot_cur_bal', 'total_rev_hi_lim', 'total_bal_ex_mort', 'all_util',
       'total_il_high_credit_limit', 'mths_since_recent_bc', 'loan_amnt',
       'total_acc', 'num_rev_accts', 'inq_last_12m', 'num_il_tl',
       'mo_sin_rcnt_rev_tl_op', 'num_bc_tl', 'open_acc', 'num_sats',
       'acc_open_past_24mths', 'mo_sin_rcnt_tl', 'pct_tl_nvr_dlq',
       'num_op_rev_tl', 'num_rev_tl_bal_gt_0', 'num_actv_rev_tl',
       'open_il_24m', 'num_tl_op_past_12m', 'num_actv_bc_tl', 'inq_fi',
       'total_cu_tl', 'total_rec_late_fee', 'open_rv_24m', 

### Easy Ensemble Classifier

In [23]:
# Train the Classifier
ee = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ee.fit(X_train_scaled, y_train)

AttributeError: 'EasyEnsembleClassifier' object has no attribute 'n_features_in_'

In [ ]:
# Calculated the balanced accuracy score
ee_y_pred = ee.predict(X_test_scaled)
balanced_accuracy_score(y_test, ee_y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, ee_y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, ee_y_pred))

### Final Questions

1. Which model had the best balanced accuracy score?



2. Which model had the best recall score?



3. Which model had the best geometric mean score?



4. What are the top three features?

